# Notebook to process pcfusage.sh script


In [ ]:
import sys
import json
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

##########################################################
# Looking to build more interactive dashboard
# TODO: Allow interactive analysis and report generation
##########################################################
import ipywidgets as widgets
from IPython.display import display

##########################################################
# Sample files capture by the pcfusage.sh script
# TODO: Add this information in a metadata section
# NOTE: Make sure the capture_date is expressed in the same
#       timezone as the foundation on which this script was
#       run or your application ages won't be correct.
##########################################################

# BORGESCLOUD File
# file = "/Users/mborges/Tools/PCF/scripts/dev_foundation.json"
# capture_date = datetime.datetime(2018, 6, 26, 0, 0)
# diego_cell = {"number_of": 4, "vcpu": 4, "ram_gb": 32, "disk_gb": 32 }

# Sample 1
# file = "/Users/mborges/Tools/PCF/scripts/samples/sample1_foundation.json"
# capture_date = datetime.datetime(2018, 7, 2, 22, 0)
# diego_cell = {"number_of": 8, "vcpu": 8, "ram_gb": 32, "disk_gb": 256 }

# Sample 2
# file = "/Users/mborges/Tools/PCF/scripts/samples/sample2_foundation.json"
# capture_date = datetime.datetime(2018, 7, 4, 22, 0)
# diego_cell = {"number_of": 6, "vcpu": 4, "ram_gb": 30.5, "disk_gb": 128 }

# Sample 3
# file = "/Users/mborges/Tools/PCF/scripts/samples/technipfmc_foundation.json"
# capture_date = datetime.datetime(2018, 7, 4, 22, 0)
# diego_cell = {"number_of": 6, "vcpu": 4, "ram_gb": 30.5, "disk_gb": 128, "operators": 0.1}

# Sample 4
# file = "/Users/mborges/Tools/PCF/scripts/samples/aa_dev_foundation.json"
# capture_date = datetime.datetime(2018, 7, 31, 22, 0)
# diego_cell = {"number_of": 31, "vcpu": 8, "ram_gb": 16, "disk_gb": 160, "operators": 2}

##########################################################
# Pre-processing of the foundation.json file
##########################################################

# Know system orgs we used to remove system applications from our analysis
system_orgs = ["system", "p-dataflow", "p-spring-cloud-services"]

# Foundation capacity from metadata section (currently data above)
total_vcpu = diego_cell['number_of'] * diego_cell['vcpu']
total_memory = diego_cell['number_of'] * diego_cell['ram_gb']

# Foundation json capture via script
with open(file, "r") as read_file:
    data = json.load(read_file)
    
# Process users dictionary    
usersDict = data[5]["users"]
    
# Creating dataframes for specific sections
# TODO: Can we not rely on the index? 
df_apps = pd.DataFrame(data[0]["apps"])
df_orgs = pd.DataFrame(data[1]["orgs"])
df_service_instances = pd.DataFrame(data[2]["service_instances"])
df_services = pd.DataFrame(data[3]["services"])
df_spaces = pd.DataFrame(data[4]["spaces"])
df_users = pd.DataFrame(usersDict)

# Fix column names so we can merge
# TODO: Look inot fixing the pcfusage.sh script so this is not longer need
df_orgs = df_orgs.rename(columns = {"name":"org"})
df_spaces = df_spaces.rename(columns = {"name": "space", "org":"org_guid"})
df_apps = df_apps.rename(columns = {"space": "space_guid"})

# Merge spaces and orgs and create an non_system_spaces series
environments = pd.merge(df_spaces, df_orgs, on="org_guid")
non_system_spaces = (environments['org'].isin(system_orgs))

# Merge apps with spaces to get org and space names
apps = pd.merge(df_apps, environments, on="space_guid")

apps = apps[apps.state != "STOPPED"]
#display(apps)

# consolidating buildpacks
java_searchfor = ['client-certificate-mapper', 'container-', 'java', 'tc-']
mask = apps['buildpack'].str.contains('|'.join(java_searchfor))
apps.loc[mask, 'buildpack'] = 'java_buildpack_offline'

dotnet_searchfor = ['netcore', 'dotnet', 'Core']
mask = apps['buildpack'].str.contains('|'.join(dotnet_searchfor))
apps.loc[mask, 'buildpack'] = 'dotnet-core_buildpack'

static_searchfor = ['static', 'siteminder']
mask = apps['buildpack'].str.contains('|'.join(static_searchfor))
apps.loc[mask, 'buildpack'] = 'staticfile_buildpack'

mask = apps['buildpack'].str.contains('node')
apps.loc[mask, 'buildpack'] = 'nodejs_buildpack'

mask = apps['buildpack'].str.contains('php')
apps.loc[mask, 'buildpack'] = 'php_buildpack'


# Merge services with service instances
services = pd.merge(df_services, df_service_instances, on="service_guid")

# Non system applications
#non_system_apps = (apps['org'] != 'system') & (apps['org'] != 'p-dataflow')
system_apps = apps['org'].isin(system_orgs)
non_system_apps = ~apps['org'].isin(system_orgs)

# This is the core dataframe for applications
df_non_system_apps = apps[non_system_apps]

# Array of user objects
# usersDict

##########################################################
# Functions
##########################################################

#d = datetime.datetime.strptime('2018-02-07T03:17:13Z', '%Y-%m-%dT%H:%M:%SZ')
def get_days(last_updated):
    deltatime = capture_date - datetime.datetime.strptime(last_updated, '%Y-%m-%dT%H:%M:%SZ')
    return deltatime.days


# Foundation by Numbers
## speed
Organizations can represent different business units, spaces environments where initial setup lead and process time are usually high. Services and buildpacks represents the middleware required to develop applications.

In [ ]:
print ("Orgs: {}".format(len(df_non_system_apps['org_guid'].unique())))
print ("Spaces: {}".format(len(df_non_system_apps['space_guid'].unique())))
print ("Users: {}".format(len(df_users)))
print ("Services: {}".format(len(df_services['label'].unique())))
print ("Unique Applications: {}".format(len(df_non_system_apps['name'].unique())))
print ("Buildpacks: {}".format(len(df_non_system_apps['buildpack'].unique())))
print ("Containers: {}".format(df_non_system_apps['instances'].sum()))
print ("Service Instances: {}".format(len(df_service_instances['name'].unique())))



# Application Instance Memory in MB
## scalability
Statistics on memory usage by all customer applications. This can show scalability of PCF by comparing min and max memory sizes. 

In [ ]:
print ("{} unique non system apps using a total Memory: {}".format(df_non_system_apps['name'].nunique(),df_non_system_apps['memory'].sum()))
df_non_system_apps['memory'].describe()


# Density
## savings
Number of containers per CPU and per vCPU. This can show infrastructure consolidation and savings

In [ ]:
containers_per_cell = apps[non_system_apps]['instances'].sum() / diego_cell['number_of']
containers_per_vcpu = apps[non_system_apps]['instances'].sum() / total_vcpu
print ("Total system apps: {} and non system apps: {}".format(apps[system_apps]['instances'].sum(), apps[non_system_apps]['instances'].sum()))
print ("containers_per_cell of non system apps {}".format(containers_per_cell))
print ("containers_per_vcpu of non system apps {}".format(containers_per_vcpu))

# Application per Orgs
This could provide a view into the different business units on the platform based on the nomenclature used. The more BUs the more compound value the platform can bring.

In [ ]:
import pylab as plt
#orgs = apps.groupby("org").size()
orgs = apps[non_system_apps].groupby("org").size()
print(orgs)


# TODO merge this with days
print(df_non_system_apps.groupby("org")['space', 'name', 'buildpack'].nunique())

pd.options.mode.chained_assignment = None
df_non_system_apps['days'] = df_non_system_apps['updated'].apply(get_days)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_non_system_apps.groupby(["org","space", "name", "buildpack"]).agg({'days': np.mean}))

orgs.plot.barh(figsize=(15,15))

plt.gca().spines["top"].set_visible(False)  
plt.gca().spines["right"].set_visible(False)

plt.ylabel('')
plt.xlabel('Applications', fontsize=16)
plt.ylabel('Organizations', fontsize=16)
plt.xticks(fontsize=14)  
plt.yticks(fontsize=14)


In [ ]:
spaces = apps[non_system_apps].groupby("space").size()
spaces.plot.barh(figsize=(15,15))

plt.gca().spines["top"].set_visible(False)  
plt.gca().spines["right"].set_visible(False)

plt.ylabel('')
plt.xlabel('Applications', fontsize=16)
plt.ylabel('Spaces', fontsize=16)
plt.xticks(fontsize=14)  
plt.yticks(fontsize=14)

# Developer Productivity - Application vs Days
## speed
Here we explore when was an application last updated comparing with the date the data was capture to infer speed. We should see an increase of deployments due to the automation of the platform. We can possibly measure developer productivity by comparing with how long releases took in the past or testing methodologies. 

In [ ]:
from matplotlib.patches import Rectangle



# df_non_system_apps['days'] = df_non_system_apps['updated'].apply(get_days)
app_days = df_non_system_apps['updated'].apply(get_days)


#app_days.plot.hist(bins=50)
N, bins, patches = plt.hist(app_days, 30, ec='k')

cmap = plt.get_cmap('jet')
hot = cmap(0.9)
warm =cmap(0.7)
cold = cmap(0.2)

for i in range(0,7):
    patches[i].set_facecolor(hot)
for i in range(7,14):
    patches[i].set_facecolor(warm)
for i in range(14,30):
    patches[i].set_facecolor(cold)

#create legend
handles = [Rectangle((0,0),1,1,color=c,ec="k") for c in [hot,warm, cold]]
labels= ["hot","warm", "cold"]
plt.legend(handles, labels)

plt.xlabel('Days', fontsize=16)
plt.ylabel('Applications', fontsize=16)
plt.xticks(fontsize=14)  
plt.yticks(fontsize=14)

plt.gca().spines["top"].set_visible(False)  
plt.gca().spines["right"].set_visible(False)



In [ ]:
app_days.describe()

# Buildpack Distribution
This provides an few of the languages and frameworks used in the platform. This can be used in regards to saving on middleware license and support. There is also day 2 ops concepts on maintaining current versions and consistently patching across environments. This is usually a subset of the available buildpack on PCF. We can talk about innovation and enabling multi-language for modern microservices based development.

In [ ]:
pd.DataFrame(df_non_system_apps.groupby("buildpack").size())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches

buildpacks = df_non_system_apps.groupby("buildpack").size()

# buildpacks.plot.pie(figsize=(20,20), title="Buildpack Distribution")
buildpacks.plot.pie(figsize=(15,15), autopct='%1.1f%%')

plt.ylabel('')


In [ ]:
AI_memory = df_apps["memory"].sum()
AI_count = df_apps["instances"].sum()
unique_apps = len(apps['name'].unique())

print("Total Apps is {}".format(unique_apps))
print("Total AIs: {} consuming {} MB RAM".format(AI_count, AI_memory))


# Services
Fast provisioning of data services, platform services like autoscaler, SSO or batch are often not easy to manage. The concept of a marketplace is huge for developer velocity who usually have to wait to get access to RDBMS or messaging systems. A lot of value to explor here.

In [ ]:

pd.DataFrame(df_service_instances.groupby("service_guid").size())

service_usage = services.groupby('label').size()

service_usage.plot.pie(figsize=(10,10))


plt.ylabel('')

In [ ]:
service_usage

# Interactive Analysis
This is an attempt to create interactive dahsboards so we can better explorer value to various customers

In [ ]:
df_apps.head()

from ipywidgets import interactive
 
items = ['All']+sorted(apps['org'].unique().tolist())
 
def view(x=''):
    if x=='All': 
        display(apps)
    else:
        result = apps[apps['org']==x] 
        print ("Total Apps: {}".format(result['instances'].count()))
        display(result)

w = widgets.Select(options=items)
widgets.VBox([widgets.Label('Organizations'), interactive(view, x=w)])




